In [12]:
import os
import requests
import pandas as pd
import random
import math

# Parameters
csv_file = "seamounts.csv"  # Path to the .csv file
output_folder = "no_seamounts_galore"  # Folder to save downloaded images
tile_pixels = 800  # Width and height of the image in pixels
tile_width_deg = 1.0  # Fixed width in degrees
exclusion_radius_deg = 2.0  # Exclusion zone around each point (in degrees)
api_base_url = "https://www.gmrt.org/services/ImageServer"
max_attempts = 10  # Max number of attempts to find valid points

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load CSV file with the points to avoid
try:
    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} records from {csv_file}.")
except Exception as e:
    print(f"Error loading .csv file: {e}")
    exit()

# Create a list of points to avoid (exclusion zones)
exclude_points = [(row["LAT"], row["LONG"]) for index, row in df.iterrows()]
print(f"Exclusion zones defined for {len(exclude_points)} points.")

# Function to generate random points outside exclusion zones
def generate_random_point():
    # Generate random latitudes and longitudes
    lat = random.uniform(-90, 90)
    lon = random.uniform(-180, 180)
    
    # Check if the point is within any exclusion zone
    for excl_lat, excl_lon in exclude_points:
        distance = math.sqrt((lat - excl_lat)**2 + (lon - excl_lon)**2)
        if distance < exclusion_radius_deg:
            return generate_random_point()  # Recursively try again if within exclusion zone
    
    return lat, lon

# Function to download the image tile
def download_tile(center_lat, center_lon, file_name):
    # Adjust height based on Mercator projection
    adjusted_height_deg = tile_width_deg * math.cos(math.radians(center_lat))
    
    # Calculate tile bounds
    minlatitude = center_lat - (adjusted_height_deg / 2)
    maxlatitude = center_lat + (adjusted_height_deg / 2)
    minlongitude = center_lon - (tile_width_deg / 2)
    maxlongitude = center_lon + (tile_width_deg / 2)

    # Prepare API parameters
    params = {
        "minlatitude": minlatitude,
        "maxlatitude": maxlatitude,
        "minlongitude": minlongitude,
        "maxlongitude": maxlongitude,
        "width": tile_pixels,
        "height": tile_pixels,  # Ensure the image is square
        "mask": "false",
        "download": "true",
    }

    # Construct file name
    output_file = os.path.join(output_folder, f"{file_name}.png")

    # Download the image
    print(f"Downloading square image for {file_name} at {center_lat}, {center_lon}...")
    response = requests.get(api_base_url, params=params)

    if response.status_code == 200:
        # Save the image
        with open(output_file, "wb") as file:
            file.write(response.content)
        print(f"Saved: {output_file}")
    else:
        print(f"Failed to download for {file_name} - HTTP {response.status_code}")

# Generate random points and download tiles
attempts = 0
tiles_downloaded = 0

while attempts < max_attempts:
    try:
        # Generate a random point outside the exclusion zones
        random_lat, random_lon = generate_random_point()
        file_name = f"tile_{random_lat:.1f}_{random_lon:.1f}"

        # Download the image tile for this point
        download_tile(random_lat, random_lon, file_name)

        tiles_downloaded += 1
        print(f"Downloaded {tiles_downloaded} tile(s).")

        # Stop if enough tiles are downloaded
        if tiles_downloaded >= 1000:
            break

    except Exception as e:
        print(f"Error during attempt {attempts}: {e}")
        attempts += 1

print(f"Download complete. {tiles_downloaded} tiles downloaded.")


Loaded 33452 records from seamounts.csv.
Exclusion zones defined for 33452 points.
Saved: no_seamounts_galore/tile_88.2_106.8.png
Downloaded 1 tile(s).
Saved: no_seamounts_galore/tile_-75.8_165.4.png
Downloaded 2 tile(s).
Saved: no_seamounts_galore/tile_75.3_27.2.png
Downloaded 3 tile(s).
Saved: no_seamounts_galore/tile_59.8_86.9.png
Downloaded 4 tile(s).
Saved: no_seamounts_galore/tile_12.4_-133.9.png
Downloaded 5 tile(s).
Saved: no_seamounts_galore/tile_73.5_16.8.png
Downloaded 6 tile(s).
Saved: no_seamounts_galore/tile_59.0_23.1.png
Downloaded 7 tile(s).
Saved: no_seamounts_galore/tile_20.0_27.0.png
Downloaded 8 tile(s).
Saved: no_seamounts_galore/tile_64.8_21.4.png
Downloaded 9 tile(s).
Saved: no_seamounts_galore/tile_-73.8_125.8.png
Downloaded 10 tile(s).
Saved: no_seamounts_galore/tile_-89.0_-75.0.png
Downloaded 11 tile(s).
Saved: no_seamounts_galore/tile_34.5_10.3.png
Downloaded 12 tile(s).
Saved: no_seamounts_galore/tile_34.5_45.7.png
Downloaded 13 tile(s).
Saved: no_seamounts_